# Добро пожаловать в мир дифференцируемых функций

**Мотивация**. Нейросети отличаются от других методов тем, у них функции ошибки всегда «гладкие» — если чуть-чуть изменить значение отдельного параметра, то вывод сети почти не изменится.

**Производная**. Интуитивно, производной называют скорость изменения функции. Её можно записать так:

$$ f'(x) = \frac{f(x+\Delta x) - f(x)}{\Delta x}, \quad \Delta x \to 0 $$

Чуть формальнее вам объяснят на курсе матана. Её в основном считают, пользуясь этим определением. Например для функции $x^2$ она такая:

$$ f'(x) = (x^2)' = \frac{(x+\Delta x)^2 - x^2}{\Delta x} = \frac{2x\Delta x + \Delta x^2}{\Delta x} = 2x + \Delta x \to 2x $$

**Градиент**. Градиентом какой-либо функции называют специальный вектор, составленный из производных по разным параметрам.

Как найти минимум такой функции? Давайте сделаем много маленьких шагов против градиента. Рано или поздно мы придем в минимум, хотя бы локальный.

В принципе, мы бы могли считать градиент численно — просто сделать для каждого параметра это маленькие изменение и посмотреть, насколько стало лучше. Это возможно, но просто *безумно* долго: для каждого параметра нам нужно прогнать все ещё раз.

Придумали способ посчитать градиент эффективно — за один прогон суммарно — основывающийся на следующем свойстве производной:

$$ f(g(x))' = f'(g(x)) g'(x) $$

TODO: доказать его

Как этот факт можно использовать? Вспомним, что сеть — это вычислительный граф. Как конкретный параметр влияет на функцию потерь? Ну, он куда-то дальше передается. Точнее, он передается последовательно в $n$ функций дальше:

$$ f_x = f_1(f_2(\ldots f_n(x))) $$

Производной такой штуки будет, соответственно,

$$ f'_x = f_1'(x) f_2(\ldots f_n(x))) $$

TODO: как-нибудь нормально записать

Получается, мы можем *рекурсивно* посчитать градиент следующих вершин в графе, и после этого мы сразу можем посчитать градиент текущей вершины.

## Стохастическая версия

Обратим внимание, что мы делаем *маленький* шаг. Но данных у нас много. Чтобы для всех вычислить градиент, потребуется много ресурсов.

Нам не обязательно считать все — достаточно взять какой-нибудь кусок данных (его называют batch) — и посчитать градиент только на нем. Градиент получается достаточно хорошим, если batch достаточно большой и репрезентативный.

## Фреймворки

У нас 12 дней, а не 6 лет, поэтому мы не будем вдаваться в подробности, как эти производные считаются аналитически. Это не очень сложно, но довольно трудоемко. Это сделали за нас.

Все фреймворки сейчас разделяются на 2 типа: статические (TensorFlow, Theano) и динамические (PyTorch, Chainer). Также есть обертки над ними (Keras), которые упрощают жизнь и абстрагируют целые слои и процедуры.

Выбор фреймворка — настоящая религиозная война. Автор пишет на PyTorch, но так как курс у нас не очень долгий, то будет писать все на Keras.

# MNIST

Есть стандартный датасет. В нем 50000 картинок, по 5000 на каждую цифру от 0 до 9. Требуется распознать их.

In [1]:
import keras
from keras.layers import Dense # «плотный» слой — матричное умножение
from keras.models import Sequential # вспомогательный класс, позволяющий последовательно выполнять операции
from keras.optimizers import SGD # Stochastic Gradient Descent

Using TensorFlow backend.


In [4]:
from keras.datasets import mnist # датасет настолько известный, что он есть по умолчанию почти везде
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11485184/11490434 [============================>.] - ETA: 0s

Особенности формата: сейчас каждая картинка представляет собой трехмерные массивы (напомним, что многомерные массивы называют тензорами) размера n x 28 x 28. Мы хотим представить каждую как вектор размера $784 = 28^2$.

In [12]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

Ещё одна деталь: инициализация весов — отдельное искусство. В фреймворках оно опять же сделано за нас. Но сделано оно так, что предполагает, что входные данные — числа от 0 до 1, а сейчас они от 0 до 255. Исправим это:

In [13]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Какой лосс использовать для классификации?

Есть такой принцип максимального правдоподобия.

Давайте максимизировать произведение вероятностей истинных событий. Звучит логично.

$$ L = \prod p_i $$

Произведение оптимизировать очень не удобно. Давайте воспользуемся следующим трюком: давайте возьмем логарифм (любой, ведь все логарифмы отличаются в константу раз) и будем максимизировать сумму:

$$ \log L = \log \prod p_i = \sum \log p_i $$

Эту штуку называют кроссэнтропией. Такое название пошло из теории информации, но нам пока знать это не надо.

Для удобноства вместо чисел — от 0 до 9 — сконвертируем их в вектора размера 10, где будет стоять единица в нужном месте, и тогда функция потерь запишется так:

...

Такая кодировка называется one-hot.

In [14]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Давайте сохраним этот пайплайн. Он нам понадобится.

In [ ]:
# save mnist

Собственно, вот в чем прелесть Keras: не нужно думать о размерах матриц, например.

In [15]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax'),
])

Есть специальная функция, которая позволит проверить, что мы все сделали правильно.

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 476,490
Trainable params: 476,490
Non-trainable params: 0
_________________________________________________________________


Keras оборачивает *статические* фреймворки. Им нужно скомпилировать сеть, заранее передав функцию потерь, оптимизатор и, опционально, желаемые метрики.

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 21s - loss: 0.5642 - acc: 0.8455 - val_loss: 0.2770 - val_acc: 0.9207
Epoch 2/10
60000/60000 [==============================] - 19s - loss: 0.2381 - acc: 0.9312 - val_loss: 0.1936 - val_acc: 0.9423
Epoch 3/10
60000/60000 [==============================] - 19s - loss: 0.1811 - acc: 0.9479 - val_loss: 0.1683 - val_acc: 0.9474
Epoch 4/10
60000/60000 [==============================] - 20s - loss: 0.1451 - acc: 0.9584 - val_loss: 0.1379 - val_acc: 0.9583
Epoch 5/10
60000/60000 [==============================] - 20s - loss: 0.1205 - acc: 0.9655 - val_loss: 0.1172 - val_acc: 0.9651
Epoch 6/10
60000/60000 [==============================] - 19s - loss: 0.1026 - acc: 0.9701 - val_loss: 0.1082 - val_acc: 0.9666
Epoch 7/10
60000/60000 [==============================] - 20s - loss: 0.0888 - acc: 0.9743 - val_loss: 0.0982 - val_acc: 0.9691
Epoch 8/10
60000/60000 [==============================

Попробуйте поиграться с параметрами сети. Попробуйте поставить другую функцию потерь, чтобы убедиться, что кроссэнтропия действительно лучше всех коррелирует с точностью.